<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/Integrations/Langchain/Agents/Retrieval_QA_with_Conversation_memory.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>

# Retrieval QA using Clarifai Vectorstore with Conversation memory.

The notebook walks you through the high level concept and idea to build a chatbot Doc Q/A in using clarifai as vectorstore and langchain's conversationbuffermemory. This enables the user to have a context of recent chat history in buffer as well as helping the user to query about document stored in the DB.

### Setup

In [ ]:
#install necessary packages
!pip install langchain
!pip install clarifai

Initialize your PAT key as environment variable.

In [ ]:
#Note we also have an option to pass the PAT key directly while calling the classes, so user can either intialize it as env variable or arguments.
import os
os.environ["CLARIFAI_PAT"]="YOUR_CLARIFAI_PAT"

*Note: Guide to get your [PAT](https://docs.clarifai.com/clarifai-basics/authentication/personal-access-tokens)*

### Import necessary libraries.

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Clarifai as Clarifaivectorstore
from langchain.llms import Clarifai as Clarifaillm
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

### Preparing Docs for ingestion into our vectorstore.

In [ ]:
loader = TextLoader("../Data/rule_book.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Initialize the Clarifai Vector DB using your PAT, app id and user id. You can give any number of docs for the Top K document retrieval value.

In [ ]:
clarifai_vector_db = Clarifaivectorstore.from_documents(
    user_id="user_name",
    app_id= "app_name",
    documents=docs,
    number_of_docs=3,
    pat="YOUR_CLARIFAI_PAT"
)

### Initialize a llm model through clarifaiLLM class.
- We are creating a langchain's conversationalbuffer memory and a retrieal QA chain.
- So basically the retrieval QA chain acts as an agent here and fetches the document when the query is related to contents from document.

https://python.langchain.com/docs/integrations/llms/clarifai

You can use several language models from [clarifai](https://clarifai.com/explore/models?filterData=%5B%7B%22field%22%3A%22use_cases%22%2C%22value%22%3A%5B%22llm%22%5D%7D%5D&page=1&perPage=24) platform.

In [ ]:
MODEL_URL = "https://clarifai.com/openai/chat-completion/models/GPT-4"

llm=Clarifaillm(model_url=MODEL_URL)

### ConversationBufferWindowMemory

- ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. **It only uses the last K interactions.**
- This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

To know more on ConversationBufferWindowMemory: https://python.langchain.com/docs/modules/memory/types/buffer_window

The **K we are using here sets the bufferwindow for chat memory**, whereas the no_of_documents we set in the previous stage in clarifaivectorstore is to retrieve top_k docs based on the query from vectorDB.


In [ ]:
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

retrieve_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=clarifai_vector_db.as_retriever()
)

Testing whether our Retrieval QA chain is working perfectly.

In [ ]:
retrieve_chain.run("How automobile classes has been segregated with respect to engines?")

'Automobile classes have been segregated with respect to engines as follows:\n\n1. F1 Grand Prix cars with 3L aspirated engines built from 1970 to 1972. This class is further divided into cars equipped with a Ford-Cosworth DFV Engine and those equipped with other engines.\n\n2. F1 Grand Prix cars with 3L aspirated engines built from 1973 to 1976. This class is also divided into cars equipped with a Ford-Cosworth DFV Engine and those equipped with other engines.\n\n3. F1 Grand Prix cars with 3L aspirated engines built from 1977 to 1980. This class is divided into cars designed not to exploit the ground effect, cars designed to exploit the ground effect equipped with a Ford-Cosworth DFV engine, and cars designed to exploit the ground effect equipped with other engines.\n\n4. Cars equipped with a Ford-Cosworth DFV engine.\n\n5. F1 Grand Prix cars 3L aspirated built from 1981 to 1985.\n\n6. Cars equipped with other engines.\n\nAdditionally, there is a Special Invitation Class for any Formu

## Clarifai Vectorstore as Tool
Now the task is to make this retrieval QA chain into something like a chatbot which also provides a conversational experience with previous messages contexts.

https://python.langchain.com/docs/integrations/vectorstores/clarifai

In [ ]:
#initialize an Tool with our clarifaivectorstore as retrieval agent.
from langchain.agents import Tool

tools = [
    Tool(
        name='Guide document',
        func=retrieve_chain.run,
        description=(
            'use this tool when answering about automobile related queries especially related to F1-70s rulebook chnages to get '
            'more information about the topic'
        )
    )
]

# Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.


More on Agents: https://python.langchain.com/docs/modules/agents/

## Conversational Agent

- This agent is designed to be used in conversational settings. 
- The prompt is designed to make the agent helpful and conversational. 
- It uses the ReAct framework to decide which tool to use, and uses memory to remember the previous conversation interactions.

Refer: https://python.langchain.com/docs/modules/agents/agent_types/chat_conversation_agent

To put all pieces together, we are building a chatbot with a conversational_memory as well a retrieval agent which fetches relevant docs to chat with.

In [ ]:
from langchain.agents import initialize_agent

chat_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method='generate',
    handle_parsing_errors=True,
    memory=conversational_memory
)

## Observations
Now we execute the some queries to understand the functionality and usage of this agent.

In [ ]:
chat_agent.run("How automobile classes has been segregated with respect to engines?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Guide document",
    "action_input": "Automobile classes and engine segregation"
}
```
Observation: The automobile classes and engine segregation mentioned in the text can be summarized as follows:

1. F1 Grand Prix cars with a 3L aspirated engine built from 1977 to 1980. These are further divided into:
   - Cars designed not to exploit the ground effect.
   - Cars designed to exploit the ground effect, equipped with a Ford-Cosworth DFV engine.
   - Cars designed to exploit the ground effect, equipped with other engines.

2. Cars equipped with a Ford-Cosworth DFV engine. 

3. F1 Grand Prix cars with a 3L aspirated engine built from 1981 to 1985. 

4. Cars equipped with other engines. 

5. Class Special Invitation: This class is for any Formula One cars or any other cars considered by the Organization to be of Special Historical interest to the Organization or Promoters of any of the races, or of particular benefit to the 

"The segregation of automobile classes with respect to engines, as per the information obtained, can be detailed as follows: \n\n1. F1 Grand Prix cars with a 3L aspirated engine built from 1977 to 1980 are divided into: \n   - Cars not designed to exploit the ground effect. \n   - Cars designed to exploit the ground effect, equipped with a Ford-Cosworth DFV engine. \n   - Cars designed to exploit the ground effect, equipped with other engines. \n\n2. There's a class for cars equipped with a Ford-Cosworth DFV engine. \n\n3. Another class is for F1 Grand Prix cars with a 3L aspirated engine built from 1981 to 1985. \n\n4. There's also a class for cars equipped with other engines. \n\n5. The 'Class Special Invitation' is for any Formula One cars or any other cars considered to be of Special Historical interest to the Organization or Promoters of any of the races, or of particular benefit to the Organization."

#### ***So when we re-instructed the bot to perform similar task without any context it can able to digest it and produce the results as expected.***

In [ ]:
chat_agent.run("Just summarize the classes")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Automobile classes, especially in the context of Formula 1 racing, are typically segregated based on engine specifications and design characteristics. Here's a brief summary:

1. Class for F1 Grand Prix cars (1977-1980) not designed to exploit the ground effect.
2. Class for F1 Grand Prix cars (1977-1980) designed to exploit the ground effect, equipped with a Ford-Cosworth DFV engine.
3. Class for F1 Grand Prix cars (1977-1980) designed to exploit the ground effect, equipped with other engines.
4. Class for cars equipped with a Ford-Cosworth DFV engine.
5. Class for F1 Grand Prix cars (1981-1985).
6. Class for cars equipped with other engines.
7. Special Invitation Class: reserved for any Formula One cars or any other cars considered to be of Special Historical interest or of particular benefit to the Organization."
}
```

> Finished chain.


"Automobile classes, especially in the context of Formula 1 racing, are typically segregated based on engine specifications and design characteristics. Here's a brief summary:\n\n1. Class for F1 Grand Prix cars (1977-1980) not designed to exploit the ground effect.\n2. Class for F1 Grand Prix cars (1977-1980) designed to exploit the ground effect, equipped with a Ford-Cosworth DFV engine.\n3. Class for F1 Grand Prix cars (1977-1980) designed to exploit the ground effect, equipped with other engines.\n4. Class for cars equipped with a Ford-Cosworth DFV engine.\n5. Class for F1 Grand Prix cars (1981-1985).\n6. Class for cars equipped with other engines.\n7. Special Invitation Class: reserved for any Formula One cars or any other cars considered to be of Special Historical interest or of particular benefit to the Organization."

In [ ]:
chat_agent.run("summarize the new rules from the document")



> Entering new AgentExecutor chain...
```json
{
    "action": "Guide document",
    "action_input": "summarize the new rules from the document"
}
```
Observation: The document outlines rules related to protests, appeals, and safety and discipline in a race event. Protests must be written and submitted within 30 minutes after the provisional classification, and they require a fee. The exact fee amount will be stated in an Event Bulletin. However, appeals cannot be made against decisions concerning several items, including a Drive Through Penalty, a Timed Stop & Go Penalty, any Time Penalty, decisions taken by the Stewards in relation to Article 23.4.2, penalties under Article 27.7 or Article 30.2, and decisions made by timekeepers and other Judges of Fact. 

In terms of general safety and discipline, the Clerk of the Course can request additional formation laps if conditions require. If additional laps are necessary, the start is considered to have been given at the end of the first f

"The document you referenced outlines rules related to protests, appeals, and safety and discipline in race events. Protests must be written and submitted within 30 minutes following the provisional classification, and they require a fee, the amount of which will be stated in an Event Bulletin. However, appeals cannot be made against decisions concerning several items, including a Drive Through Penalty, a Timed Stop & Go Penalty, any Time Penalty, decisions taken by the Stewards in relation to certain articles, penalties under specific articles, and decisions made by timekeepers and other Judges of Fact. \n\nIn terms of general safety and discipline, the Clerk of the Course can request additional formation laps if conditions require. If additional laps are necessary, the start is considered to have been given at the end of the first formation lap. The document also suggests there are specific cases in which a variation in the start procedure is allowed, but these cases aren't specified

#### ***Okay, What if when we asks some random question to our agent ?.***

In [ ]:
chat_agent.run("what is story of big bang theory")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The Big Bang Theory is a cosmological model that describes the universe's observable development from the earliest known periods through its subsequent large-scale evolution. The model describes how the universe expanded from an extremely high-density and high-temperature state, and offers a comprehensive explanation for a broad range of observed phenomena, including the abundance of light elements, the cosmic microwave background (CMB) radiation, and the large scale structure of the cosmos. The Big Bang Theory does not explain why the universe was created or what occurred before the event itself, as these questions fall into the realm of philosophy or theology."
}
```

> Finished chain.


"The Big Bang Theory is a cosmological model that describes the universe's observable development from the earliest known periods through its subsequent large-scale evolution. The model describes how the universe expanded from an extremely high-density and high-temperature state, and offers a comprehensive explanation for a broad range of observed phenomena, including the abundance of light elements, the cosmic microwave background (CMB) radiation, and the large scale structure of the cosmos. The Big Bang Theory does not explain why the universe was created or what occurred before the event itself, as these questions fall into the realm of philosophy or theology."

#### ***So in above context our agent just did a action observation by itself and found that it doesn't need the agent to answer for the query and went ahead straight to generate response for the queries.***

In [ ]:
chat_agent.run("I meant the series story")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The Big Bang Theory is an American television sitcom that aired from 2007 to 2019. It was created by Chuck Lorre and Bill Prady. The show is primarily centered around five characters living in Pasadena, California: Leonard Hofstadter and Sheldon Cooper, both physicists at Caltech, who share an apartment; Penny, a waitress and aspiring actress who later becomes a pharmaceutical representative and who lives across the hall; and Leonard and Sheldon's similarly geeky and socially awkward friends and co-workers, aerospace engineer Howard Wolowitz and astrophysicist Raj Koothrappali. The geekiness and intellect of the four men are contrasted for comic effect with Penny's social skills and common sense. Over time, supporting characters were promoted to starring roles, including neuroscientist Amy Farrah Fowler, microbiologist Bernadette Rostenkowski, physicist Leslie Winkle and comic book store owner 

"The Big Bang Theory is an American television sitcom that aired from 2007 to 2019. It was created by Chuck Lorre and Bill Prady. The show is primarily centered around five characters living in Pasadena, California: Leonard Hofstadter and Sheldon Cooper, both physicists at Caltech, who share an apartment; Penny, a waitress and aspiring actress who later becomes a pharmaceutical representative and who lives across the hall; and Leonard and Sheldon's similarly geeky and socially awkward friends and co-workers, aerospace engineer Howard Wolowitz and astrophysicist Raj Koothrappali. The geekiness and intellect of the four men are contrasted for comic effect with Penny's social skills and common sense. Over time, supporting characters were promoted to starring roles, including neuroscientist Amy Farrah Fowler, microbiologist Bernadette Rostenkowski, physicist Leslie Winkle and comic book store owner Stuart Bloom."

#### ***So even when we re-instructed the prompt with a different query but in the same context, it was able to perceive it since we have our chat memory in conversation and produce results based on that.***

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---